In [1]:
import sys

# Add the parent directory to the system path
sys.path.append("../04_survival_models/src")

In [2]:
import os
from collections import Counter

import mlflow
from azureml.core import Workspace, Experiment
from azure.ai.ml import command, Input, MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.entities import AmlCompute
from uc2_functions import *
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
import shutil
from tqdm import tqdm
import mltable

# Goal

The goal is to run the script located at `PATH_SCRIPT` multiple times with different seeds (Monte Carlo simulations). This selection drives diverse train-test data shuffling and initializes imputers and models, ensuring full reproducibility of experiments.

# Parameters

In [3]:
subscription_id = "753a0b42-95dc-4871-b53e-160ceb0e6bc1"
resource_group = "rg-s-race-aml-dev-we"
workspace_name = "amlsraceamldevwe01"
cpu_compute_target = "clusteramldev01"

EXPERIMENT_NAME_SELECTOR = "UC2_raw_2024_02" # Used to get top features
EXPERIMENT_NAME = "UC2_raw_survival_models_2024_07"
PATH_SCRIPT = "04_survival_models_raw_csm.py"
DIR_MODEL_PKL = "../models_pkl"
DIR_ARTIFACTS = "artifacts"
DIR_SRC = "src"
DIR_EXPERIMENTS = "experiments"
#PATH_CONFIG = "mock.json"
PATH_CONFIG = "config.json"
PATH_IMPORTANCES = "df_importances.json"

# Init client

In [4]:
workspace = Workspace(subscription_id, resource_group, workspace_name)
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

Found the config file in: /config.json


# Sample random numbers between 0 and 1000

Same seeds as feature selection phase

In [5]:
random_numbers = [0,
 1,
 6,
 8,
 23,
 25,
 27,
 30,
 32,
 40,
 42,
 62,
 73,
 89,
 90,
 91,
 95,
 104,
 114,
 129,
 136,
 142,
 160,
 163,
 166,
 178,
 200,
 203,
 207,
 209,
 217,
 223,
 225,
 228,
 237,
 238,
 250,
 255,
 269,
 281,
 284,
 342,
 367,
 376,
 379,
 391,
 394,
 395,
 429,
 432,
 433,
 457,
 459,
 460,
 462,
 517,
 533,
 535,
 539,
 551,
 554,
 558,
 574,
 586,
 592,
 597,
 603,
 604,
 616,
 619,
 654,
 665,
 667,
 692,
 694,
 697,
 704,
 718,
 733,
 734,
 754,
 755,
 758,
 759,
 771,
 775,
 790,
 805,
 818,
 825,
 826,
 828,
 885,
 890,
 914,
 932,
 963,
 968,
 975,
 996]

In [6]:
print(len(random_numbers))

100


# Get top features from feature selectors

In [7]:
workspace = Workspace.from_config()

# Check if the experiment exists, if not, create it
if EXPERIMENT_NAME_SELECTOR not in workspace.experiments:
    experiment = Experiment(workspace, EXPERIMENT_NAME_SELECTOR)
else:
    experiment = workspace.experiments[EXPERIMENT_NAME_SELECTOR]

# Set the MLflow tracking URI to point to your Azure ML workspace
mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())
client = mlflow.tracking.MlflowClient()

In [8]:
os.makedirs(DIR_ARTIFACTS, exist_ok=True)
l = []
gen = experiment.get_runs(include_children=True)
for run in tqdm(gen):
    # Access the run in MLflow
    data = mlflow.get_run(run.id).data
    # Check if child run
    if ("mlflow.parentRunId" in data.tags) and ("random_state" in data.params) and (int(data.params['random_state']) in random_numbers):
        # Get model path
        artifacts = mlflow.tracking.MlflowClient().list_artifacts(run.id)
        # Dowload feature importance
        if "RandomSurvivalForest_selector_" in data.tags["mlflow.runName"]:
            client.download_artifacts(
                run_id=run.id, path="df_importance_rsf", dst_path=DIR_ARTIFACTS
            )
            df_importance_i = pd.read_json(os.path.join(DIR_ARTIFACTS, "df_importance_rsf")).sort_values("importances_mean", ascending=False)
            df_importance_i = df_importance_i[df_importance_i['importances_mean'] > 0]
            d = dict()
            d['model'] = data.tags["mlflow.runName"]
            d['random_state'] = int(data.params['random_state'])
            d['top_features'] = df_importance_i['importances_mean'].keys().tolist()
            l.append(d)
        else:
            continue

df_importances = pd.DataFrame(l)
df_importances.sort_values("random_state", ascending=False)
# Write for all seeds
df_importances.to_json(path_or_buf=os.path.join(DIR_SRC, DIR_EXPERIMENTS, PATH_IMPORTANCES), orient="records")
del l, d, df_importance_i
shutil.rmtree(DIR_ARTIFACTS)

0it [00:00, ?it/s]

1it [00:01,  1.96s/it]

4it [00:03,  1.57it/s]

5it [00:04,  1.28it/s]

8it [00:06,  1.70it/s]

...

392it [03:50,  2.30it/s]

393it [03:51,  1.95it/s]

396it [03:52,  2.18it/s]

397it [03:53,  1.83it/s]

400it [03:55,  1.70it/s]


# Exclude seeds with previous training

Some seeds may have previous simulations logged on mlflow, we don't need to retrain those simulations.

In [9]:
workspace = Workspace.from_config()

# Check if the experiment exists, if not, create it
if EXPERIMENT_NAME not in workspace.experiments:
    experiment = Experiment(workspace, EXPERIMENT_NAME)
else:
    experiment = workspace.experiments[EXPERIMENT_NAME]

# Set the MLflow tracking URI to point to your Azure ML workspace
mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())
client = mlflow.tracking.MlflowClient()

In [10]:
l = []
gen = experiment.get_runs(include_children=True)
for run in tqdm(gen):
    # Access the run in MLflow
    data = client.get_run(run.id).data
    # Check if child run
    if "mlflow.parentRunId" in data.tags:
        l.append(data.params["random_state"])
    else:
        continue
l = sorted([int(x) for x in l])

1485it [06:59,  3.54it/s]


In [11]:
# Check if each unique value appears exactly n_models times
problematic_seeds = find_problematic_values(l, 14)
assert problematic_seeds == []

# Seeds where to start training from scratch
random_states = sorted([x for x in random_numbers if x not in list(set(l))])
print(len(random_states), "simulations left")

1 simulations left


# Init cluser

In [12]:
cpu_cluster = ml_client.compute.get(cpu_compute_target)
print(f"AMLCompute with name {cpu_cluster.name}, the compute size is {cpu_cluster.size}")

AMLCompute with name clusteramldev01, the compute size is STANDARD_D2_V2


# Set dataset

In [13]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("UC2_raw_survival_csm_ohe_5yrs", version="23")

Found the config file in: /config.json


# Set mlflow

In [14]:
mlflow_tracking_uri =str(ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri)
run_description = "Survival models UC2"

# Run jobs

Run the same training script multiple times changing seeds

In [15]:
for random_state in tqdm(random_states):
    display_name = str(random_state) + "_" + PATH_CONFIG.split('/')[-1].split('.')[0]
    inputs = dict(RANDOM_STATE=random_state,
                    EXPERIMENT_NAME=EXPERIMENT_NAME,
                    DATA_ID=data_asset.id,
                    PATH_IMPORTANCES=os.path.join(DIR_EXPERIMENTS, PATH_IMPORTANCES),
                    N_MAX_FEATURES=16,
                    PATH_CONFIG=os.path.join(DIR_EXPERIMENTS, PATH_CONFIG),
                    DIR_MODEL_PKL=DIR_MODEL_PKL,
                    )
    formatted_keys = [f'--{k} \'{json.dumps(inputs[k]) if isinstance(inputs[k], dict) else inputs[k]}\'' for k in inputs.keys()]
    py_command = "python {} ".format(PATH_SCRIPT) + ' '.join(formatted_keys)
    # configure job
    job = command(
        inputs=inputs,
        code=DIR_SRC,
        command=py_command,
        environment='uc2_pat:5',
        compute=cpu_compute_target,
        experiment_name=EXPERIMENT_NAME,
        display_name="S_{}".format(random_state),
        instance_count=1)
    # submit job
    returned_job = ml_client.create_or_update(job)

  0%|          | 0/1 [00:00<?, ?it/s]Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.

